In [1]:
from matplotlib import pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime
import os
from tqdm import tqdm_notebook as tqdm
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas

In [40]:
def get_lemma_distribution(file_name):
    lemmas=[]
    pattern = re.compile('http://en.wikipedia.org/wiki/[a-z,A-z,_]+$')
    for line in open(file_name):
        url=line.strip().split(' ')[2]
        if not pattern.match(url):
            continue
        lemma=url[url.rfind('/')+1:]
        if '.' in lemma:
            continue
        lemmas.append(lemma)
    return lemmas

In [41]:
visit_count=dict()
f=open('./data/list2.txt').readlines()[1:]
for line in tqdm(f[:100]):
    line=line.strip()[:-3]
    lemmas=get_lemma_distribution('./data/only_lemma/%s'%line)
    for entry in lemmas:
        try:
            visit_count[entry]+=1
        except:
            visit_count[entry]=1

In [42]:
visit_count_df=pandas.DataFrame(list(visit_count.items()),columns=['Entry','Visit Count'])

In [43]:
visit_count_df=visit_count_df.to_csv('./entry_visit_count.csv',index=False)

In [44]:
visit_count_df=pandas.read_csv('./entry_visit_count.csv')

In [45]:
visit_count_df=visit_count_df.dropna(axis=0, how='any')

In [47]:
entry_len=visit_count_df['Entry'].map(len)

In [48]:
entry_len_Q1,entry_len_Q3,entry_len_iqr=get_iqr(entry_len.values)

In [49]:
entry_len_Q1,entry_len_Q3,entry_len_iqr

(10.0, 19.0, 9.0)

In [51]:
visit_count_df=visit_count_df[entry_len<entry_len_Q3+1.5*entry_len_iqr]

In [71]:
visit_count_df.head()

,Entry,Visit Count
0,Karen_Sparck_Jones,1
1,Bathybates,2
2,Picquot,1
3,Another_Sun,8
4,Serge_LeClerc,5


In [72]:
visit_count_df['Visit Count'].mean()

18.135140001193875